In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib.colors import Normalize

In [ ]:
type(fashion_mnist) # fashion_mnist is a module inside tensorflow.keras.datasets

module

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
#The function load_data() returns a tuple containing training and testing data

# Print the shape of data
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (60000, 28, 28)
Test data shape: (10000, 28, 28)


In [ ]:
np.max(X_train)

255

In [ ]:
y_train # contains numbers (0–9), each representing a clothing category.

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
np.unique(y_train) # returns the sorted unique class labels present in y_train

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
class_names = [
    "t-shirt/top", "trouser", "pullover", "dress", "coat",
    "sandal", "shirt", "sneaker", "bag", "ankle boot"
]
print(class_names) 

# 0 → t-shirt/top  
# 1 → trouser  
# 2 → pullover  
# 3 → dress  
# 4 → coat  
# 5 → sandal  
# 6 → shirt  
# 7 → sneaker  
# 8 → bag  
# 9 → ankle boot  


['t-shirt/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']


4. Data Exploration

4.1 Dimensions of the training data

In [ ]:
X_train.shape 

# 60000 → Number of training images (60,000 samples)
# 28 → Image height (28 pixels)
# 28 → Image width (28 pixels)

In [ ]:
X_test.shape

# 10000 → Number of test images (10,000 samples)
# 28 → Image height (28 pixels)
# 28 → Image width (28 pixels)

In [ ]:
plt.figure() # Creates a new figure in Matplotlib
plt.imshow(X_train[0]) # Displays the second image from X_train
plt.colorbar() # Adds a color scale next to the image.

In [ ]:
y_train

In [ ]:
X_train = X_train/255.0

In [ ]:
X_test = X_test/255.0

In [ ]:
np.max(X_train) # Max pixel value in the dataset X_train

In [ ]:
plt.figure()
plt.imshow(X_train[0])
plt.colorbar()

5 Data Augmentation

In [ ]:
# Reshape for augmentation (adding a channel dimension)

X_train = X_train.reshape(-1, 28, 28, 1)

In [ ]:
# Select 5 random images

random_indices = random.sample(range(X_train.shape[0]), 5)
random_images = X_train[random_indices]

In [ ]:
# 5 image augmentation techniques
augmentations = {
    "Rotated (30°)": ImageDataGenerator(rotation_range=30),
    "Zoomed (20%)": ImageDataGenerator(zoom_range=0.2),
    "Width Shifted (20%)": ImageDataGenerator(width_shift_range=0.2),
    "Height Shifted (20%)": ImageDataGenerator(height_shift_range=0.2),
    "Sheared (15°)": ImageDataGenerator(shear_range=15),
}

In [ ]:
from matplotlib.colors import Normalize

fig, axes = plt.subplots(len(random_images), len(augmentations) + 1, figsize=(12, 12))

# Choose a colormap - you can change this to other options like 'jet', 'plasma', 'inferno', etc.
colormap = 'viridis'

# Create a normalizer to ensure consistent color mapping across all images
# Assuming pixel values are between 0-1 for grayscale images
norm = Normalize(vmin=0, vmax=1)

for i, image in enumerate(random_images):
    # Make sure image is 2D by squeezing if needed
    image_2d = image.squeeze()
    
    # Display the original image with colormap in the first column
    axes[i, 0].imshow(image_2d, cmap=colormap, norm=norm)
    axes[i, 0].set_title("Original")
    axes[i, 0].axis("off")
    
    # Apply each augmentation and display
    for j, (title, generator) in enumerate(augmentations.items()):
        # Prepare image for augmentation
        if len(image.shape) == 2:
            # Add batch and channel dimensions: (28,28) → (1,28,28,1)
            image_for_aug = np.expand_dims(np.expand_dims(image, axis=0), axis=-1)
        elif len(image.shape) == 3 and image.shape[-1] == 1:
            # Just add batch dimension: (28,28,1) → (1,28,28,1)
            image_for_aug = np.expand_dims(image, axis=0)
        else:
            # Handle other cases if needed
            image_for_aug = np.expand_dims(image, axis=0)
            
        # Generate augmented image
        augmented_image = next(generator.flow(image_for_aug, batch_size=1))[0]
        
        # Make sure augmented image is 2D
        augmented_image_2d = augmented_image.squeeze()
        
        # Display augmented image with colormap
        axes[i, j + 1].imshow(augmented_image_2d, cmap=colormap, norm=norm)
        axes[i, j + 1].set_title(title)
        axes[i, j + 1].axis("off")

plt.tight_layout()
plt.show()

## 6. Training Models


### 6.1 Model Training on Original Data  without Augmentation

In [ ]:
# Reshape data to add channel dimension (for CNN)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [ ]:
# Convert labels to categorical format (for categorical_crossentropy loss)
y_train_categorical = to_categorical(y_train, num_classes=10)
y_test_categorical = to_categorical(y_test, num_classes=10)

In [ ]:
def create_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model

# Create and compile model BEFORE augmentation
model_before = create_model()
model_before.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_before.summary()

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_before = model_before.fit(
    X_train, y_train_categorical, 
    epochs=5, batch_size=64,
    validation_data=(X_test, y_test_categorical), 
  
)





In [ ]:
model_before.save("model_before_augmentation.keras")

## 6.2  Model Training on Aug. Data 

In [ ]:
# Define data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=5, 
    zoom_range=0.1,
    width_shift_range=0.05, 
    height_shift_range=0.05, 
    horizontal_flip=True
)
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

# Create augmented data generator
train_generator = train_datagen.flow(X_train, y_train_categorical, batch_size=64)

# Create and compile model AFTER augmentation
model_after = create_model()  # Create a new instance of the model
model_after.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with augmented data
history_after = model_after.fit(
    train_generator, 
    epochs=5,
    validation_data=(X_test, y_test_categorical),
   
)



### Saving Model after augmentation

In [ ]:
model_after.save("model_after_augmentation.keras")